In [2]:
import pandas as pd

# Load the CSV
df = pd.read_csv("urls.csv")

# Split into chunks of 1000
chunk_size = 1000

for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i + chunk_size]
    chunk.to_csv(f"urls_part_{i//chunk_size + 1}.csv", index=False)

print("Done! Files created.")

Done! Files created.


In [ ]:



from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait    # added
from selenium.webdriver.support import expected_conditions as EC #added
import csv  # <-- added for saving results


def scrape_properties(urls, output_file="properties.csv"):  # <-- added function definition
    # Firefox options (headless)
    options = Options()
    options.add_argument("--headless")
    options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
 

    # Prepare CSV file
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["url", "property_id", "property_type", "locality_name", "postal_code", "build_year", "state" ,"furnished", "number_rooms","equipped_kitchen", "terrace,facades", "garden", "garden_area","swimming_pool", "price"])

        # Loop through URLs
        for url in urls:
            print(url)
            driver = None
            try:
                # Launch Firefox
                GECKO_PATH = r"C:\users\geckodriver.exe"  # <-- your local path
                service = Service(GECKO_PATH)
                driver = webdriver.Firefox(service=service, options=options)
                #print(f"\n Scraping {i} of {len(urls)} → {url}")
                driver.get(url)

                # ---- Handle cookie consent automatically ----
                
                try:
                    # Wait for the "Agree and close" button to be clickable
                    accept_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    accept_button.click()
                    print("Cookies accepted.")
                except Exception as e:
                    print("No cookie popup found or could not click the accept button:", e)
                # ---- End cookie handling ----

                
                ## TERRACE: 1 = Yes, 0 = No
                try:
                    terrace_tag = driver.find_element(By.CSS_SELECTOR, "h4:contains('Terrace') + p")
                    terrace_text = terrace_tag.text.strip()
                    terrace = 1 if "yes" in terrace_text.lower() else 0
                except:
                    terrace = None 

                # GARDEN: 1 = Yes, 0 = No
                try:
                    garden_blocks = driver.find_elements(By.CSS_SELECTOR, "div")
                    garden = 0  # default
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "garden" == h4_text:  # exact match for Garden
                                garden_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden = 1 if "yes" in garden_text.lower() else 0
                                break
                        except:
                            continue
                except:
                    garden = 0

                # GARDEN AREA (m²): integer or None
                try:
                    garden_area = None
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "surface garden" in h4_text:
                                garden_area_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
                                break
                        except:
                            continue
                except:
                    garden_area = None
                try:
                    facades_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of facades')]/following-sibling::p").text
                    facades = int(facades_text)
                except:
                    facades = None


                    ################################  :-)
                # SWIMMING POOL: 1 = Yes, 0 = No
                try:
                    pool_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Swimming pool')]/following-sibling::p").text
                    swimming_pool = 1 if "Yes" in pool_text else 0
                except:
                    swimming_pool = 0


                ###########################"" ADDING MULTIPLE ITESM
                # STATE OF BUILDING: text or None
                try:
                    state = driver.find_element(By.XPATH, "//h4[contains(text(), 'State of the property')]/following-sibling::p").text
                except:
                    state = None

                # BUILD YEAR: integer or None 
                try:
                    build_year_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Build Year')]/following-sibling::p").text
                    build_year = int(build_year_text)
                except:
                    build_year = None

                # NUMBER OF ROOMS: integer or None
                try:
                    number_rooms_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of bedrooms')]/following-sibling::p").text
                    number_rooms = int(number_rooms_text)
                except:
                    number_rooms = None

                    # LIVING AREA (m²): integer or None
                try:
                    living_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Livable surface')]/following-sibling::p").text
                    living_area = int(living_area_text.replace("m²", "").replace("m", "").strip())
                except:
                    living_area = None

                # EQUIPPED KITCHEN: 1 = Yes, 0 = No, None if missing 
            
                try:
                    kitchen_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Kitchen equipment')]/following-sibling::p").text
                    if "equipped" in kitchen_text.lower():  # any text containing "equipped"
                        equipped_kitchen = 1
                    elif "not" in kitchen_text.lower():      # text containing "not"
                        equipped_kitchen = 0
                    else:
                        equipped_kitchen = None
                except:
                    equipped_kitchen = None

                # FURNISHED: 1 = Yes, 0 = No, None if missing
                try:
                    furnished_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Furnished')]/following-sibling::p").text
                    if "yes" in furnished_text.lower():
                        furnished = 1
                    elif "no" in furnished_text.lower():
                        furnished = 0
                    else:
                        furnished = None
                except:
                        furnished = None

                ###########################"" MULTIPLE ITEMS ADDED ABOVE  
                
                

                # -----------------------
                # Extract property ID
                # -----------------------
                try:
                    property_id = url.split('/')[-1]
                except:
                    property_id = None

                # -----------------------
                # Extract postal code & locality
                # -----------------------
                try:
                    city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
                    city_text = city_tag.text.strip()
                    postal_code = city_text.split()[0]
                    locality_name = " ".join(city_text.split()[1:])
                except:
                    postal_code = None
                    locality_name = None

                # -----------------------
                # Extract price
                # -----------------------
                try:
                    price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
                    price = price_tag.text.strip()
                except:
                    price = None

                # -----------------------
                # Extract property type & subtype
                # -----------------------
                try:
                    type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
                    type_words = type_tag.text.strip().split()
                    property_type = type_words[0]
                except:
                    property_type = None

                # -----------------------
                # Print results
                # -----------------------
                print("Terrace:", terrace)
                print("Garden presence:", garden)
                print("Garden area:", garden_area)
                print("Number of facades:", facades)
                print("Swimming pool:", swimming_pool)
                print("State of building:", state)
                print("Build year:", build_year)
                print("Number of rooms:", number_rooms)
                print("Living area (m²):", living_area)
                print("Equipped kitchen:", equipped_kitchen)
                print("Furnished:", furnished)
                print("Property ID:", property_id)
                print("Postal Code:", postal_code)
                print("Locality:", locality_name)
                print("Price:", price)
                print("Type:", property_type)
                

                # Write row to CSV file
                
                row = [
                        url, property_id, property_type, locality_name, postal_code,
                        build_year, state, furnished, number_rooms, equipped_kitchen,
                        terrace, facades, garden, garden_area, swimming_pool, price
                    ]

                # Replace empty strings or None with literal "None"
                cleaned_row = [item if (item not in [None, ""]) else "None" for item in row]

                writer.writerow(cleaned_row)
            except Exception as e:
                print(f"Error loading page: {url}")
                print("Reason:", e)
            finally:
                if driver:
                    driver.quit()




# -----------------------------
# ✅ USE SECTION 
# -----------------------------
input_file = "urls_part_1.csv"
output_file = "properties-1rst.csv"

urls = []

# Read URLs from the second column (with header Type,URL)
with open(input_file, mode="r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if "URL" in row and row["URL"].startswith("http"):
            urls.append(row["URL"])

print(f"Loaded {len(urls)} URLs from {input_file}")

# Run the scraper
scrape_properties(urls, output_file)
print(f"✅ Scraping complete. Results saved in '{output_file}'")



Loaded 1000 URLs from urls_part_1.csv
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbs97482
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: New
Build year: 2017
Number of rooms: 1
Living area (m²): 60
Equipped kitchen: None
Furnished: 0
Property ID: rbs97482
Postal Code: 3500
Locality: Hasselt
Price: 189 000 €
Type: Apartment
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbu17157
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: 1979
Number of rooms: 2
Living area (m²): 90
Equipped kitchen: None
Furnished: 0
Property ID: rbu17157
Postal Code: 3500
Locality: Hasselt
Price: 197 000 €
Type: Apartment
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbu24804
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
Stat